# ORNL AUC Calculation Method #
We utilise the Eagle-I dataset and the sliced FEMA declaration information

In [44]:
# Import necessary libraries
import pandas as pd
import numpy as np
from scipy.integrate import trapezoid
#from google.colab import drive

# Mount Google Drive
#drive.mount('/content/drive')

# Define the directory path for data in Google Drive
#data_path = '/content/drive/My Drive/DATA/'  # Adjust the path as needed

# Load outage data for years 2017 to 2023
ea17csv = pd.read_csv("E:/DATA/outage/eagle-i/county/eaglei_outages_2017.csv")
ea18csv = pd.read_csv("E:/DATA/outage/eagle-i/county/eaglei_outages_2018.csv")
ea19csv = pd.read_csv("E:/DATA/outage/eagle-i/county/eaglei_outages_2019.csv")
ea20csv = pd.read_csv("E:/DATA/outage/eagle-i/county/eaglei_outages_2020.csv")
ea21csv = pd.read_csv("E:/DATA/outage/eagle-i/county/eaglei_outages_2021.csv")
ea22csv = pd.read_csv("E:/DATA/outage/eagle-i/county/eaglei_outages_2022.csv")
ea23csv = pd.read_csv("E:/DATA/outage/eagle-i/county/eaglei_outages_2023.csv")

# Combine all years' data into a single DataFrame
outages = pd.concat([ea17csv, ea18csv, ea19csv, ea20csv, ea21csv, ea22csv, ea23csv], ignore_index=True)

# Remove NA values
outages = outages.dropna()

# Extract data for specific regions
tolland = outages[outages['fips_code'] == 9013]
ga_out = outages[(outages['fips_code'] >= 13000) & (outages['fips_code'] <= 13999)]
ms_out = outages[(outages['fips_code'] >= 28000) & (outages['fips_code'] <= 28999)]

tolland

,fips_code,county,state,sum,run_start_time
11039,9013,Tolland,Connecticut,1.0,2017-01-01 13:15:00
11286,9013,Tolland,Connecticut,1.0,2017-01-01 13:30:00
11548,9013,Tolland,Connecticut,1.0,2017-01-01 13:45:00
11830,9013,Tolland,Connecticut,1.0,2017-01-01 14:00:00
12129,9013,Tolland,Connecticut,1.0,2017-01-01 14:15:00
...,...,...,...,...,...
159679215,9013,Tolland,Connecticut,5.0,2023-12-30 20:00:00
159679787,9013,Tolland,Connecticut,5.0,2023-12-30 20:15:00
159680355,9013,Tolland,Connecticut,5.0,2023-12-30 20:30:00
159680930,9013,Tolland,Connecticut,5.0,2023-12-30 20:45:00


In [16]:
ga_out

,fips_code,county,state,sum,run_start_time
38,13003,Atkinson,Georgia,1.0,2017-01-01 00:00:00
39,13021,Bibb,Georgia,2.0,2017-01-01 00:00:00
40,13063,Clayton,Georgia,1.0,2017-01-01 00:00:00
41,13067,Cobb,Georgia,1.0,2017-01-01 00:00:00
42,13089,DeKalb,Georgia,73.0,2017-01-01 00:00:00
...,...,...,...,...,...
159729436,13209,Montgomery,Georgia,2.0,2023-12-31 23:45:00
159729437,13267,Tattnall,Georgia,1.0,2023-12-31 23:45:00
159729438,13283,Treutlen,Georgia,5.0,2023-12-31 23:45:00
159729439,13301,Warren,Georgia,4.0,2023-12-31 23:45:00


This is a markdown too

In [35]:
# Process `tolland` DataFrame for outage blocks, begin and end times, outage ID, and AUC calculation
tolland['run_start_time'] = pd.to_datetime(tolland['run_start_time'])
tolland = tolland.sort_values('run_start_time').reset_index(drop=True)
tolland['time_diff'] = tolland['run_start_time'].diff().dt.total_seconds() / 60
tolland['new_outage'] = np.where((tolland['time_diff'].isna()) | (tolland['time_diff'] > 15), 1, 0)
tolland['outage_ID'] = tolland['new_outage'].cumsum()
tolland


# Process `tolland` DataFrame for outage blocks, begin and end times, outage ID, and AUC calculation
ga_out['run_start_time'] = pd.to_datetime(ga_out['run_start_time'])
ga_out = ga_out.sort_values(['fips_code','run_start_time']).reset_index(drop=True)
ga_out['time_diff'] = ga_out['run_start_time'].diff().dt.total_seconds() / 60
ga_out['new_outage'] = np.where((ga_out['time_diff'].isna()) | (ga_out['time_diff'] > 15), 1, 0)
ga_out['outage_ID'] = ga_out['new_outage'].cumsum()
ga_out

,fips_code,county,state,sum,run_start_time,time_diff,new_outage,outage_ID
0,13001,Appling,Georgia,49.0,2017-01-01 13:30:00,NaN,1,1
1,13001,Appling,Georgia,49.0,2017-01-01 13:45:00,15.0,0,1
2,13001,Appling,Georgia,49.0,2017-01-01 14:00:00,15.0,0,1
3,13001,Appling,Georgia,49.0,2017-01-01 14:15:00,15.0,0,1
4,13001,Appling,Georgia,49.0,2017-01-01 14:30:00,15.0,0,1
...,...,...,...,...,...,...,...,...
7705298,13321,Worth,Georgia,1.0,2023-12-31 15:45:00,15.0,0,678314
7705299,13321,Worth,Georgia,1.0,2023-12-31 16:00:00,15.0,0,678314
7705300,13321,Worth,Georgia,1.0,2023-12-31 23:15:00,435.0,1,678315
7705301,13321,Worth,Georgia,1.0,2023-12-31 23:30:00,15.0,0,678315


In [36]:
# Grouped calculations for each outage block in `tolland`
tolland_grouped = tolland.groupby(['fips_code', 'outage_ID'])
outage_begin, outage_end, cumulative_time, cumulative_custm, auc_list = [], [], [], [], []

for name, group in tolland_grouped:
    start_time = group['run_start_time'].iloc[0]
    end_time = group['run_start_time'].iloc[-1]
    time_elapsed = (group['run_start_time'] - start_time).dt.total_seconds() / 60
    cumulative_sum = group['sum'].cumsum()
    auc_value = trapezoid(y=[0] + cumulative_sum.tolist(), x=[0] + time_elapsed.tolist())
    
    outage_begin.extend([start_time] * len(group))
    outage_end.extend([end_time] * len(group))
    cumulative_time.extend(time_elapsed + 15)
    cumulative_custm.extend(cumulative_sum)
    auc_list.extend([auc_value] * len(group))

# Add calculated columns to `tolland`
tolland['outage_begin'] = outage_begin
tolland['outage_end'] = outage_end
tolland['cumulative_time'] = cumulative_time
tolland['cumulative_custm'] = cumulative_custm
tolland['auc'] = auc_list
tolland


# Grouped calculations for each outage block in `ga_out`
ga_out_grouped = ga_out.groupby(['fips_code', 'outage_ID'])
outage_begin, outage_end, cumulative_time, cumulative_custm, auc_list = [], [], [], [], []

for name, group in ga_out_grouped:
    start_time = group['run_start_time'].iloc[0]
    end_time = group['run_start_time'].iloc[-1]
    time_elapsed = (group['run_start_time'] - start_time).dt.total_seconds() / 60
    cumulative_sum = group['sum'].cumsum()
    auc_value = trapezoid(y=[0] + cumulative_sum.tolist(), x=[0] + time_elapsed.tolist())
    
    outage_begin.extend([start_time] * len(group))
    outage_end.extend([end_time] * len(group))
    cumulative_time.extend(time_elapsed + 15)
    cumulative_custm.extend(cumulative_sum)
    auc_list.extend([auc_value] * len(group))

# Add calculated columns to `ga_out`
ga_out['outage_begin'] = outage_begin
ga_out['outage_end'] = outage_end
ga_out['cumulative_time'] = cumulative_time
ga_out['cumulative_custm'] = cumulative_custm
ga_out['auc'] = auc_list
ga_out



,fips_code,county,state,sum,run_start_time,time_diff,new_outage,outage_ID,outage_begin,outage_end,cumulative_time,cumulative_custm,auc
0,13001,Appling,Georgia,49.0,2017-01-01 13:30:00,NaN,1,1,2017-01-01 13:30:00,2017-01-01 16:15:00,15.0,49.0,52552.5
1,13001,Appling,Georgia,49.0,2017-01-01 13:45:00,15.0,0,1,2017-01-01 13:30:00,2017-01-01 16:15:00,30.0,98.0,52552.5
2,13001,Appling,Georgia,49.0,2017-01-01 14:00:00,15.0,0,1,2017-01-01 13:30:00,2017-01-01 16:15:00,45.0,147.0,52552.5
3,13001,Appling,Georgia,49.0,2017-01-01 14:15:00,15.0,0,1,2017-01-01 13:30:00,2017-01-01 16:15:00,60.0,196.0,52552.5
4,13001,Appling,Georgia,49.0,2017-01-01 14:30:00,15.0,0,1,2017-01-01 13:30:00,2017-01-01 16:15:00,75.0,245.0,52552.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7705298,13321,Worth,Georgia,1.0,2023-12-31 15:45:00,15.0,0,678314,2023-12-31 14:45:00,2023-12-31 16:00:00,75.0,5.0,262.5
7705299,13321,Worth,Georgia,1.0,2023-12-31 16:00:00,15.0,0,678314,2023-12-31 14:45:00,2023-12-31 16:00:00,90.0,6.0,262.5
7705300,13321,Worth,Georgia,1.0,2023-12-31 23:15:00,435.0,1,678315,2023-12-31 23:15:00,2023-12-31 23:45:00,15.0,1.0,60.0
7705301,13321,Worth,Georgia,1.0,2023-12-31 23:30:00,15.0,0,678315,2023-12-31 23:15:00,2023-12-31 23:45:00,30.0,2.0,60.0


In [38]:
ga_out

,fips_code,county,state,sum,run_start_time,time_diff,new_outage,outage_ID,outage_begin,outage_end,cumulative_time,cumulative_custm,auc
0,13001,Appling,Georgia,49.0,2017-01-01 13:30:00,NaN,1,1,2017-01-01 13:30:00,2017-01-01 16:15:00,15.0,49.0,52552.5
1,13001,Appling,Georgia,49.0,2017-01-01 13:45:00,15.0,0,1,2017-01-01 13:30:00,2017-01-01 16:15:00,30.0,98.0,52552.5
2,13001,Appling,Georgia,49.0,2017-01-01 14:00:00,15.0,0,1,2017-01-01 13:30:00,2017-01-01 16:15:00,45.0,147.0,52552.5
3,13001,Appling,Georgia,49.0,2017-01-01 14:15:00,15.0,0,1,2017-01-01 13:30:00,2017-01-01 16:15:00,60.0,196.0,52552.5
4,13001,Appling,Georgia,49.0,2017-01-01 14:30:00,15.0,0,1,2017-01-01 13:30:00,2017-01-01 16:15:00,75.0,245.0,52552.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7705298,13321,Worth,Georgia,1.0,2023-12-31 15:45:00,15.0,0,678314,2023-12-31 14:45:00,2023-12-31 16:00:00,75.0,5.0,262.5
7705299,13321,Worth,Georgia,1.0,2023-12-31 16:00:00,15.0,0,678314,2023-12-31 14:45:00,2023-12-31 16:00:00,90.0,6.0,262.5
7705300,13321,Worth,Georgia,1.0,2023-12-31 23:15:00,435.0,1,678315,2023-12-31 23:15:00,2023-12-31 23:45:00,15.0,1.0,60.0
7705301,13321,Worth,Georgia,1.0,2023-12-31 23:30:00,15.0,0,678315,2023-12-31 23:15:00,2023-12-31 23:45:00,30.0,2.0,60.0


This is markdown

In [ ]:
# Consolidate `tolland_avg` DataFrame
tolland_avg = (
    tolland.groupby(['fips_code', 'outage_ID'], as_index=False)
    .agg(
        fipsCode=('fips_code', 'first'),
        outage_ID=('outage_ID', 'first'),
        avg=('sum', 'mean'),
        outage_begin=('outage_begin', 'first'),
        outage_end=('outage_end', 'first'),
        duration=('cumulative_time', 'last'),
        tot_custm=('cumulative_custm', 'last'),
        auc=('auc', 'first')
    )
)

# Calculate auc_nrm
tolland_avg['auc_nrm'] = tolland_avg.apply(
    lambda row: row['auc'] / row['tot_custm'] if row['tot_custm'] != 0 else np.nan, axis=1
)

# Select only the required columns
tolland_avg = tolland_avg[['fipsCode', 'outage_ID', 'avg', 'outage_begin', 'outage_end', 'duration', 'tot_custm', 'auc', 'auc_nrm']]

tolland_avg


# Consolidate `tolland_avg` DataFrame
ga_out_avg = (
    ga_out.groupby(['fips_code', 'outage_ID'], as_index=False)
    .agg(
        fipsCode=('fips_code', 'first'),
        outage_ID=('outage_ID', 'first'),
        avg=('sum', 'mean'),
        outage_begin=('outage_begin', 'first'),
        outage_end=('outage_end', 'first'),
        duration=('cumulative_time', 'last'),
        tot_custm=('cumulative_custm', 'last'),
        auc=('auc', 'first')
    )
)

# Calculate auc_nrm
ga_out_avg['auc_nrm'] = ga_out_avg.apply(
    lambda row: row['auc'] / row['tot_custm'] if row['tot_custm'] != 0 else np.nan, axis=1
)

# Select only the required columns
ga_out_avg = ga_out_avg[['fipsCode', 'outage_ID', 'avg', 'outage_begin', 'outage_end', 'duration', 'tot_custm', 'auc', 'auc_nrm']]

ga_out_avg

,fipsCode,outage_ID,avg,outage_begin,outage_end,duration,tot_custm,auc,auc_nrm
0,13001,1,49.000000,2017-01-01 13:30:00,2017-01-01 16:15:00,180.0,588.0,52552.5,89.375000
1,13001,2,3.000000,2017-01-02 04:00:00,2017-01-02 04:00:00,15.0,3.0,0.0,0.000000
2,13001,3,7.857143,2017-01-03 06:30:00,2017-01-03 09:45:00,210.0,110.0,10687.5,97.159091
3,13001,4,3.000000,2017-01-03 11:30:00,2017-01-03 12:30:00,75.0,15.0,540.0,36.000000
4,13001,5,1.000000,2017-01-03 16:00:00,2017-01-03 17:15:00,90.0,6.0,262.5,43.750000
...,...,...,...,...,...,...,...,...,...
678468,13321,678311,4.000000,2023-12-29 17:45:00,2023-12-29 17:45:00,15.0,4.0,0.0,0.000000
678469,13321,678312,1.000000,2023-12-29 19:45:00,2023-12-29 20:00:00,30.0,2.0,22.5,11.250000
678470,13321,678313,1.000000,2023-12-30 05:45:00,2023-12-30 06:00:00,30.0,2.0,22.5,11.250000
678471,13321,678314,1.000000,2023-12-31 14:45:00,2023-12-31 16:00:00,90.0,6.0,262.5,43.750000


This is markdown

In [33]:
# Read disaster declaration data
dec_1723 = pd.read_csv("E:/DATA/dec_1723.csv")

# Filter for "IDALIA" disasters and impacted counties
dec_disaster = dec_1723[dec_1723['declarationTitle'].str.contains("IDALIA", na=False)]
imp_cties = dec_disaster['fipsCode'].unique()

# Extract incident begin and end dates
begin = pd.to_datetime(dec_disaster['incidentBeginDate'].iloc[0]).date()
end = pd.to_datetime(dec_disaster['incidentEndDate'].iloc[0]).date() + pd.Timedelta(days=1)

# Convert begin and end dates to timestamps
begin = pd.to_datetime(f"{begin} 00:00:00")
end = pd.to_datetime(f"{end} 23:59:59")

dec_disaster

,Unnamed: 0,femaDeclarationString,disasterNumber,state,declarationType,declarationDate,fyDeclared,incidentType,declarationTitle,ihProgramDeclared,...,lastIAFilingDate,incidentId,region,designatedIncidentTypes,lastRefresh,hash,id,fipsCode,Declared,Requested
10,11,DR-4738-GA,4738,GA,DR,2023-09-07,2023,Hurricane,HURRICANE IDALIA,0,...,2023-11-06,2023082403,4,"2,4,F,H,T,W",2024-08-27 18:22:14.8,9fedf5304f082c05bca99aaab910e19d86655f08,00de6d40-0652-426a-a2cb-a13b5b370f6f,13001,1,1
11,12,DR-4738-GA,4738,GA,DR,2023-09-07,2023,Hurricane,HURRICANE IDALIA,0,...,2023-11-06,2023082403,4,"2,4,F,H,T,W",2024-08-27 18:22:14.8,a75f5c5ab0d7981a4c538942c77b1deccc17c021,b37beee7-8b33-46a1-97ab-86c0ba7e0c1e,13003,1,1
12,13,DR-4738-GA,4738,GA,DR,2023-09-07,2023,Hurricane,HURRICANE IDALIA,0,...,2023-11-06,2023082403,4,"2,4,F,H,T,W",2024-08-27 18:22:14.8,ed1a50cc4e0ed80d19578b2c48982d1b3e319c89,a654e178-92d3-411e-bdeb-2a3ee2273d4b,13005,1,1
13,14,DR-4738-GA,4738,GA,DR,2023-09-07,2023,Hurricane,HURRICANE IDALIA,0,...,2023-11-06,2023082403,4,"2,4,F,H,T,W",2024-08-27 18:22:14.8,dd49e0261f220c48cbcbda548017cf2badae9e54,cfb1ac05-a172-4834-84ea-347443e91f99,13025,1,1
14,15,DR-4738-GA,4738,GA,DR,2023-09-07,2023,Hurricane,HURRICANE IDALIA,0,...,2023-11-06,2023082403,4,"2,4,F,H,T,W",2024-08-27 18:22:14.8,f1909ed07f3226d25b378fc31c6531339e9d595f,24de5832-ca7e-44ce-b4ff-dbccf72a23c7,13031,1,1
15,16,DR-4738-GA,4738,GA,DR,2023-09-07,2023,Hurricane,HURRICANE IDALIA,0,...,2023-11-06,2023082403,4,"2,4,F,H,T,W",2024-08-27 18:22:14.8,e9e86494051f53ec790a3ff134fd399126ecab46,fb9d4c95-e23c-40f4-a346-13539a55e5db,13033,1,1
16,17,DR-4738-GA,4738,GA,DR,2023-09-07,2023,Hurricane,HURRICANE IDALIA,0,...,2023-11-06,2023082403,4,"2,4,F,H,T,W",2024-08-27 18:22:14.8,9604a92e0c050e3e9df4df2a8eb11fa478616cc3,c6f43377-61e1-4ae6-a6be-5a366781977b,13039,1,1
17,18,DR-4738-GA,4738,GA,DR,2023-09-07,2023,Hurricane,HURRICANE IDALIA,0,...,2023-11-06,2023082403,4,"2,4,F,H,T,W",2024-08-27 18:22:14.8,56fd510781900b4cde96a18e2a48655a91473f8e,fbb38aa7-a529-4709-a851-fe8fb6c5eac7,13043,1,1
18,19,DR-4738-GA,4738,GA,DR,2023-09-07,2023,Hurricane,HURRICANE IDALIA,0,...,2023-11-06,2023082403,4,"2,4,F,H,T,W",2024-08-27 18:22:14.8,82951326624a98377c9177ec8bc5b4b8f9e5b5d8,7898a924-aa79-4b21-8c7a-e4571f018d36,13049,1,1
19,20,DR-4738-GA,4738,GA,DR,2023-09-07,2023,Hurricane,HURRICANE IDALIA,0,...,2023-11-06,2023082403,4,"2,4,F,H,T,W",2024-08-27 18:22:14.8,c0565891c24133ee1e40862744a1475efa80aa11,3626126e-ed24-458a-b1b9-a5b968c07aeb,13065,1,1


Matching outages to disaster

In [42]:
# Match outages for IDALIA in Georgia
idalia_ga = ga_out_avg[
    (ga_out_avg['fipsCode'].isin(imp_cties)) &
    (ga_out_avg['outage_begin'] <= end) &
    (ga_out_avg['outage_end'] >= begin)
]

idalia_ga

,fipsCode,outage_ID,avg,outage_begin,outage_end,duration,tot_custm,auc,auc_nrm
5703,13001,5704,1.000000,2023-08-30 01:15:00,2023-08-30 01:30:00,30.0,2.0,2.250000e+01,11.250000
5704,13001,5705,1141.608696,2023-08-30 10:45:00,2023-09-01 20:00:00,3450.0,262570.0,6.894867e+08,2625.915584
9387,13003,9387,1092.860927,2023-08-30 11:15:00,2023-09-02 14:30:00,4530.0,330044.0,1.083450e+09,3282.745460
13549,13005,13548,3.777778,2023-08-30 00:30:00,2023-08-30 02:30:00,135.0,34.0,2.160000e+03,63.529412
13550,13005,13549,1.000000,2023-08-30 11:45:00,2023-08-30 12:45:00,75.0,5.0,1.800000e+02,36.000000
...,...,...,...,...,...,...,...,...,...
641567,13299,641421,1.000000,2023-08-30 06:15:00,2023-08-30 07:00:00,60.0,4.0,1.125000e+02,28.125000
641568,13299,641422,4196.762195,2023-08-30 10:30:00,2023-09-02 20:15:00,4920.0,1376538.0,4.774059e+09,3468.163610
653489,13305,653340,1.000000,2023-08-30 09:30:00,2023-08-30 09:30:00,15.0,1.0,0.000000e+00,0.000000
653490,13305,653341,40.214286,2023-08-30 12:15:00,2023-08-30 15:30:00,210.0,563.0,4.794000e+04,85.150977
